# 🔍 Explainable AI (XAI) - Hands-on Learning Notebook

## Course: Lecture 19 - Model Explainability

### Learning Objectives

- Understand complexity-interpretability trade-off
- Implement intrinsically interpretable models
- Apply feature importance methods
- Use model-agnostic explanations (LIME, Surrogate Models)

### Structure

1. Setup & Data
2. Exercise 1: Complexity vs Interpretability
3. Exercise 2: Linear Model Interpretation
4. Exercise 3: Decision Trees
5. Exercise 4: Permutation Importance
6. Exercise 5: Partial Dependence Plots
7. Exercise 6: ICE Plots
8. Exercise 7: Surrogate Models
9. Exercise 8: LIME Explanations

**Instructor:** Ho-min Park | homin.park@ghent.ac.kr

---

## 🔧 Setup

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.inspection import (
    permutation_importance,
    PartialDependenceDisplay
)

plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(42)
print("✅ Setup complete!")

---

## 📊 Data: Credit Risk Dataset

In [ ]:
# Create synthetic credit risk data
np.random.seed(42)
n = 1000

data = pd.DataFrame({
    'Age': np.random.randint(20, 71, n),
    'Income': np.random.gamma(3, 15, n) + 20,
    'Credit_Score': np.random.normal(650, 80, n).clip(300, 850),
    'Debt_Ratio': np.random.beta(2, 5, n),
    'Employment_Years': np.random.exponential(8, n).clip(0, 40),
    'Education': np.random.choice([0, 1, 2, 3], n, p=[0.3, 0.4, 0.2, 0.1])
})

# Target: Loan approval (based on risk score)
risk = (
    0.3 * (data['Credit_Score'] - 300) / 550 + 
    0.2 * data['Income'] / 150 + 
    0.2 * (1 - data['Debt_Ratio']) +
    0.15 * data['Employment_Years'] / 40 +
    0.15 * data['Education'] / 3
)

prob = 1 / (1 + np.exp(-5 * (risk - 0.5)))
data['Approved'] = (
    prob + np.random.normal(0, 0.1, n) > 0.5
).astype(int)

print(f"Dataset: {data.shape}")
print(f"Approval rate: {data['Approved'].mean():.1%}")
data.head()

In [ ]:
# Split data
X = data.drop('Approved', axis=1)
y = data['Approved']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train),
    columns=X.columns
)
X_test_scaled = pd.DataFrame(
    scaler.transform(X_test),
    columns=X.columns
)

print(f"Train: {len(X_train)}, Test: {len(X_test)}")

---

## Exercise 1: Complexity vs Interpretability Trade-off

### Concept

- **Simple models**: High interpretability, may underfit
- **Complex models**: High accuracy, "black boxes"

### Task

Train 4 models and visualize the trade-off

In [ ]:
# Train models
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Decision Tree (d=3)': DecisionTreeClassifier(max_depth=3),
    'Random Forest': RandomForestClassifier(n_estimators=100),
    'Neural Network': MLPClassifier(hidden_layer_sizes=(50,))
}

results = []
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    acc = accuracy_score(y_test, model.predict(X_test_scaled))
    
    interp = {
        'Logistic Regression': 0.95,
        'Decision Tree (d=3)': 0.90,
        'Random Forest': 0.40,
        'Neural Network': 0.20
    }[name]
    
    results.append({
        'Model': name,
        'Accuracy': acc,
        'Interpretability': interp
    })
    print(f"{name}: Acc={acc:.3f}, Interp={interp:.2f}")

results_df = pd.DataFrame(results)

In [ ]:
# Visualize trade-off
plt.figure(figsize=(10, 6))

for i, row in results_df.iterrows():
    plt.scatter(
        row['Interpretability'],
        row['Accuracy'],
        s=200,
        alpha=0.6
    )
    plt.annotate(
        row['Model'],
        (row['Interpretability'], row['Accuracy']),
        xytext=(5, 5),
        textcoords='offset points'
    )

plt.xlabel('Interpretability', fontsize=12, fontweight='bold')
plt.ylabel('Accuracy', fontsize=12, fontweight='bold')
plt.title(
    'Model Complexity vs Interpretability Trade-off',
    fontsize=14,
    fontweight='bold'
)
plt.grid(alpha=0.3)
plt.show()

---

## Exercise 2: Linear Model Interpretation

### Concept

Linear models: $y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + ...$

**Interpretation**:

- **Coefficient magnitude**: Feature importance
- **Sign**: Positive/negative relationship

In [ ]:
# Train logistic regression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_scaled, y_train)

# Extract coefficients
coef_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': lr.coef_[0]
}).sort_values('Coefficient')

print(coef_df)

In [ ]:
# Visualize coefficients
plt.figure(figsize=(10, 6))

colors = ['red' if c < 0 else 'green' for c in coef_df['Coefficient']]
plt.barh(
    coef_df['Feature'],
    coef_df['Coefficient'],
    color=colors,
    alpha=0.7
)

plt.xlabel('Coefficient Value', fontweight='bold')
plt.title('Feature Coefficients (Linear Model)', fontsize=14, fontweight='bold')
plt.axvline(0, color='black', linestyle='--')
plt.grid(alpha=0.3, axis='x')
plt.show()

---

## Exercise 3: Decision Tree Transparency

### Concept

Decision trees create **IF-THEN rules**:

- Easy to follow logic path
- Visualize entire tree structure
- Extract feature importance

In [ ]:
# Train decision tree
tree = DecisionTreeClassifier(max_depth=3, random_state=42)
tree.fit(X_train, y_train)

acc = accuracy_score(y_test, tree.predict(X_test))
print(f"Accuracy: {acc:.3f}")

# Feature importance
imp_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': tree.feature_importances_
}).sort_values('Importance', ascending=False)

print(imp_df)

In [ ]:
# Visualize tree
plt.figure(figsize=(20, 10))

plot_tree(
    tree,
    feature_names=X.columns,
    class_names=['Reject', 'Approve'],
    filled=True,
    rounded=True,
    fontsize=10
)

plt.title('Decision Tree (depth=3)', fontsize=16, fontweight='bold')
plt.show()

---

## Exercise 4: Permutation Importance

### Concept

Measure feature importance by:

1. Train model
2. Shuffle one feature
3. Measure performance drop
4. Importance = baseline - shuffled

In [ ]:
# Train Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Compute permutation importance
perm_imp = permutation_importance(
    rf,
    X_test,
    y_test,
    n_repeats=30,
    random_state=42
)

perm_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': perm_imp.importances_mean,
    'Std': perm_imp.importances_std
}).sort_values('Importance', ascending=False)

print(perm_df)

In [ ]:
# Visualize with error bars
plt.figure(figsize=(10, 6))

plt.barh(
    perm_df['Feature'],
    perm_df['Importance'],
    xerr=perm_df['Std'],
    color='steelblue',
    alpha=0.7
)

plt.xlabel('Importance (Performance Drop)', fontweight='bold')
plt.title('Permutation Importance', fontsize=14, fontweight='bold')
plt.grid(alpha=0.3, axis='x')
plt.show()

---

## Exercise 5: Partial Dependence Plots (PDP)

### Concept

PDP shows **marginal effect** of a feature:

- How does prediction change as we vary the feature?
- Reveals non-linear relationships
- Model-agnostic

In [ ]:
# Create PDP for key features
fig, ax = plt.subplots(figsize=(14, 10))

PartialDependenceDisplay.from_estimator(
    rf,
    X_train,
    features=['Credit_Score', 'Income', 'Debt_Ratio', 'Age'],
    kind='average',
    n_cols=2,
    ax=ax
)

plt.suptitle('Partial Dependence Plots', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

---

## Exercise 6: ICE (Individual Conditional Expectation) Plots

### Concept

ICE = PDP for **each instance separately**

- Shows heterogeneity
- Reveals subgroups
- PDP = Average of ICE curves

In [ ]:
# Create ICE plots
n_samples = 50
ice_idx = np.random.choice(X_train.index, size=n_samples, replace=False)
X_ice = X_train.loc[ice_idx]

fig, ax = plt.subplots(figsize=(14, 10))

PartialDependenceDisplay.from_estimator(
    rf,
    X_ice,
    features=['Credit_Score', 'Income', 'Debt_Ratio', 'Age'],
    kind='both',  # Shows both ICE and PDP
    n_cols=2,
    ax=ax,
    line_kw={'alpha': 0.3},
    pd_line_kw={'color': 'red', 'linewidth': 3}
)

plt.suptitle(
    f'ICE Plots ({n_samples} instances) + PDP (red)',
    fontsize=16,
    fontweight='bold'
)
plt.tight_layout()
plt.show()

---

## Exercise 7: Surrogate Models

### Concept

Approximate **black-box** with **interpretable model**:

**Process:**

1. Train complex model (RF)
2. Get predictions
3. Train simple model (DT) on those predictions
4. Interpret surrogate

In [ ]:
# Train complex model
complex_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=None
)
complex_model.fit(X_train, y_train)
complex_acc = accuracy_score(y_test, complex_model.predict(X_test))

# Generate predictions
y_train_pred = complex_model.predict(X_train)
y_test_pred = complex_model.predict(X_test)

# Train surrogate
surrogate = DecisionTreeClassifier(max_depth=5, random_state=42)
surrogate.fit(X_train, y_train_pred)  # Train on complex model predictions!

# Measure fidelity
fidelity = accuracy_score(y_test_pred, surrogate.predict(X_test))
surr_acc = accuracy_score(y_test, surrogate.predict(X_test))

print(f"Complex model accuracy: {complex_acc:.3f}")
print(f"Surrogate fidelity: {fidelity:.3f} (matches black-box {fidelity*100:.0f}% of time)")
print(f"Surrogate accuracy: {surr_acc:.3f}")

In [ ]:
# Visualize surrogate
plt.figure(figsize=(20, 12))

plot_tree(
    surrogate,
    feature_names=X.columns,
    class_names=['Reject', 'Approve'],
    filled=True,
    rounded=True,
    fontsize=10
)

plt.title(
    f'Surrogate Model (depth=5, fidelity={fidelity:.2f})',
    fontsize=16,
    fontweight='bold'
)
plt.show()

---

## Exercise 8: LIME (Local Interpretable Model-agnostic Explanations)

### Concept

Explain **individual predictions**:

1. Perturb instance
2. Get black-box predictions
3. Fit local linear model
4. Interpret coefficients

**Key**: Explains this specific instance (not global behavior)

In [ ]:
# Install LIME
try:
    import lime
    import lime.lime_tabular
except:
    import sys
    !{sys.executable} -m pip install lime --quiet
    import lime
    import lime.lime_tabular

print("✅ LIME ready")

In [ ]:
# Create LIME explainer
explainer = lime.lime_tabular.LimeTabularExplainer(
    np.array(X_train),
    feature_names=X.columns.tolist(),
    class_names=['Rejected', 'Approved'],
    mode='classification'
)

# Select instance to explain
idx = 0
instance = X_test.iloc[idx].values
pred = rf.predict([instance])[0]
prob = rf.predict_proba([instance])[0]

print(f"Instance {idx}:")
print(f"Prediction: {'Approved' if pred==1 else 'Rejected'} ({prob[pred]:.1%} confidence)")

In [ ]:
# Generate LIME explanation
explanation = explainer.explain_instance(
    instance,
    rf.predict_proba,
    num_features=6,
    num_samples=5000
)

# Extract explanation
exp_list = explanation.as_list()

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))

features = [e[0].split()[0] for e in exp_list]
contrib = [e[1] for e in exp_list]
colors = ['green' if c > 0 else 'red' for c in contrib]

plt.barh(features, contrib, color=colors, alpha=0.7)
plt.xlabel('Feature Contribution', fontweight='bold')
plt.title(
    f'LIME Explanation: Instance {idx}',
    fontsize=14,
    fontweight='bold'
)
plt.axvline(0, color='black', linestyle='--')
plt.grid(alpha=0.3, axis='x')
plt.show()

print("\nFeature Contributions:")
for feat, cont in exp_list:
    print(f"  {feat}: {cont:+.3f}")

---

## 📚 Summary

### What We Learned

1. ✅ **Trade-off**: Complexity vs Interpretability
2. ✅ **Linear Models**: Coefficient interpretation
3. ✅ **Decision Trees**: IF-THEN rules, visualization
4. ✅ **Permutation Importance**: Model-agnostic feature ranking
5. ✅ **PDP**: Marginal feature effects
6. ✅ **ICE**: Instance-level heterogeneity
7. ✅ **Surrogate Models**: Approximate black-box
8. ✅ **LIME**: Local instance explanations

### Method Selection Guide

- **Global explanation** → Permutation Importance, PDP
- **Local explanation** → LIME, ICE
- **Interpretable model** → Linear, Decision Tree
- **Black-box approximation** → Surrogate Model

### Best Practices

- Use **multiple methods**
- Validate with **domain experts**
- Document **limitations**
- Match method to **audience**

---

## 🎓 Congratulations!

You completed the XAI Hands-on Notebook!

**Next**: Apply to your own datasets

**Contact**: homin.park@ghent.ac.kr